In [52]:
corpus = ""
for No in range(0, 10):
    filepath = "trump-speeches\speeches\speech"+str(No)+".txt"
    print(filepath)
    
    try:
        f = open(filepath, "r")
        speech = f.read()
    except:
        print("no such file")
    corpus = corpus + speech

try:
    f = open("corpus.txt", "w")
    f.write(corpus)
except:
    print("unable to print to file")

trump-speeches\speeches\speech0.txt
trump-speeches\speeches\speech1.txt
trump-speeches\speeches\speech2.txt
trump-speeches\speeches\speech3.txt
trump-speeches\speeches\speech4.txt
trump-speeches\speeches\speech5.txt
trump-speeches\speeches\speech6.txt
trump-speeches\speeches\speech7.txt
trump-speeches\speeches\speech8.txt


In [54]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
# import os

# import corpus
file = open("corpus.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines) 

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces 
data = data.split()
data = ' '.join(data)
data[:500]

len(data)


tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

sequences = []

for i in range(4, len(sequence_data)):
    words = sequence_data[i-4:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

X = []
y = []

for i in sequences:
    X.append(i[0:4])
    y.append(i[4])
    
X = np.array(X)
y = np.array(y)

y = to_categorical(y, num_classes=vocab_size)
y[:5]


model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=4))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

model.summary()

from tensorflow import keras
from keras.utils.vis_utils import plot_model

keras.utils.plot_model(model, to_file='plot.png', show_layer_names=True)

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))

# adapt epochs & batch size here:
model.fit(X, y, epochs=1, batch_size=42, callbacks=[checkpoint])


4798
The Length of sequences are:  101209
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3, 10)             47980     
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 1000)           4044000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_3 (Dense)              (None, 4798)              4802798   
Total params: 17,899,778
Trainable params: 17,899,778
Non-trainable params: 0
_________________________________________________________________
('You must install pydot (`pip install pydot`) and install graphvi

KeyboardInterrupt: 

In [6]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word
  

Let's see how it works!

In [7]:
 
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      # loop over to create longer texts:
      text = text.split(" ")
      for i in range(20):
        try:
            text_in = text[-4:]
            print(text) # print text everytime you loop
          
            next = Predict_Next_Words(model, tokenizer, text_in)
            text.append(next)
            
        except Exception as e:
          print("Error occurred: ",e)
          continue

print(text)

# changed 4 to 4 words to make it a bit stronger...
# ...still runs into loops xD 
  

['All', 'Mexicans', 'are']
a
['All', 'Mexicans', 'are', 'a']
lot
['All', 'Mexicans', 'are', 'a', 'lot']
of
['All', 'Mexicans', 'are', 'a', 'lot', 'of']
the
['All', 'Mexicans', 'are', 'a', 'lot', 'of', 'the']
people
['All', 'Mexicans', 'are', 'a', 'lot', 'of', 'the', 'people']
and
['All', 'Mexicans', 'are', 'a', 'lot', 'of', 'the', 'people', 'and']
i
['All', 'Mexicans', 'are', 'a', 'lot', 'of', 'the', 'people', 'and', 'i']
said
['All', 'Mexicans', 'are', 'a', 'lot', 'of', 'the', 'people', 'and', 'i', 'said']
i
['All', 'Mexicans', 'are', 'a', 'lot', 'of', 'the', 'people', 'and', 'i', 'said', 'i']
have
['All', 'Mexicans', 'are', 'a', 'lot', 'of', 'the', 'people', 'and', 'i', 'said', 'i', 'have']
to
['All', 'Mexicans', 'are', 'a', 'lot', 'of', 'the', 'people', 'and', 'i', 'said', 'i', 'have', 'to']
be
['All', 'Mexicans', 'are', 'a', 'lot', 'of', 'the', 'people', 'and', 'i', 'said', 'i', 'have', 'to', 'be']
able
['All', 'Mexicans', 'are', 'a', 'lot', 'of', 'the', 'people', 'and', 'i', 'said

KeyboardInterrupt: Interrupted by user